In [21]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score


In [22]:
transform_lenet = transforms.Compose([
    transforms.Resize((32, 32)),  # Suitable for LeNet-5
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set_lenet = SVHN(root='./data', split='train', download=True, transform=transform_lenet)
test_set_lenet = SVHN(root='./data', split='test', download=True, transform=transform_lenet)

train_loader_lenet = DataLoader(train_set_lenet, batch_size=64, shuffle=True)
test_loader_lenet = DataLoader(test_set_lenet, batch_size=64, shuffle=False)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [23]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes for SVHN

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [24]:
def train_and_evaluate_lenet(model, train_loader, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):  # loop over the dataset multiple times
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
    print(f'Classification Report:\n{classification_report(y_true, y_pred)}')

# Initialize and Train LeNet-5
lenet_model = LeNet5()
train_and_evaluate_lenet(lenet_model, train_loader_lenet, test_loader_lenet)


Epoch 1, Loss: 0.8926209406561206
Epoch 2, Loss: 0.46027420497617344
Epoch 3, Loss: 0.38863524000717564
Epoch 4, Loss: 0.3473430911331198
Epoch 5, Loss: 0.3184801222162736
Epoch 6, Loss: 0.29405783008800324
Epoch 7, Loss: 0.2752845263832521
Epoch 8, Loss: 0.25926259597966766
Epoch 9, Loss: 0.24106461627644743
Epoch 10, Loss: 0.22749812419823162
Accuracy: 0.8860248924400738
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      1744
           1       0.90      0.95      0.93      5099
           2       0.94      0.90      0.92      4149
           3       0.81      0.87      0.84      2882
           4       0.91      0.90      0.90      2523
           5       0.94      0.81      0.87      2384
           6       0.84      0.85      0.85      1977
           7       0.92      0.88      0.90      2019
           8       0.81      0.83      0.82      1660
           9       0.81      0.87      0.84      1595

    a

In [25]:
def adapt_model_for_svhn(model):
    if isinstance(model, torchvision.models.AlexNet) or isinstance(model, torchvision.models.VGG):
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, 10)
    elif isinstance(model, torchvision.models.ResNet):
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 10)
    return model


In [26]:
transform_pretrained = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for pretrained models
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])

train_set_pretrained = SVHN(root='./data', split='train', download=True, transform=transform_pretrained)
test_set_pretrained = SVHN(root='./data', split='test', download=True, transform=transform_pretrained)

train_loader_pretrained = DataLoader(train_set_pretrained, batch_size=64, shuffle=True)
test_loader_pretrained = DataLoader(test_set_pretrained, batch_size=64, shuffle=False)


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [27]:
def train_and_evaluate_pretrained(model, train_loader, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):  # Loop over the dataset multiple times
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
    print(f'Classification Report:\n{classification_report(y_true, y_pred)}')


In [ ]:
pretrained_models = {
    'AlexNet': torchvision.models.alexnet(pretrained=True),
    'VGG16': torchvision.models.vgg16(pretrained=True),
    'ResNet18': torchvision.models.resnet18(pretrained=True),
    'ResNet50': torchvision.models.resnet50(pretrained=True),
    'ResNet101': torchvision.models.resnet101(pretrained=True)
}

for model_name, model in pretrained_models.items():
    print(f"\nTraining and evaluating {model_name}...")
    model = adapt_model_for_svhn(model)
    train_and_evaluate_pretrained(model, train_loader_pretrained, test_loader_pretrained)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  w


Training and evaluating AlexNet...
Epoch 1, Loss: 2.242017735352162
Epoch 2, Loss: 2.238747703560575
Epoch 3, Loss: 2.2380817681941405
Epoch 4, Loss: 2.2377743031780795
Epoch 5, Loss: 2.2374580102195907
Epoch 6, Loss: 2.2373317720588117
Epoch 7, Loss: 2.2372564109652324
Epoch 8, Loss: 2.2371490428541425
Epoch 9, Loss: 2.2370639342928555
Epoch 10, Loss: 2.2370372289132865
Accuracy: 0.1958743085433313
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1744
           1       0.20      1.00      0.33      5099
           2       0.00      0.00      0.00      4149
           3       0.00      0.00      0.00      2882
           4       0.00      0.00      0.00      2523
           5       0.00      0.00      0.00      2384
           6       0.00      0.00      0.00      1977
           7       0.00      0.00      0.00      2019
           8       0.00      0.00      0.00      1660
           9       0.00      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1, Loss: 2.2509748194415495
Epoch 2, Loss: 2.2400775821968977
Epoch 3, Loss: 2.2393393447826004
